In [31]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [4]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'discomfort belly_pain burping hungry'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'C:/Users/User/Desktop/Git Repo/Baby_cry_classification/b_cry/{g}'):
        songname = f'C:/Users/User/Desktop/Git Repo/Baby_cry_classification/b_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

In [5]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [7]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'discomfort belly_pain burping hungry'.split()
for g in genres:
    for filename in os.listdir(f'C:/Users/User/Desktop/Git Repo/Baby_cry_classification/b_cry/{g}'):
        songname = f'C:/Users/User/Desktop/Git Repo/Baby_cry_classification/b_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [59]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [135]:
import missingno
missingno.matrix(data)

AttributeError: 'bytes' object has no attribute 'shape'

In [60]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [92]:
classifier = model.fit(X_train,y_train,epochs=50,batch_size=128)

Epoch 1/50
3/3 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 1.00 - 0s 8ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 2/50
3/3 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 1.00 - 0s 6ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 3/50
3/3 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 1.00 - 0s 9ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 4/50
3/3 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 1.00 - 0s 12ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 5/50
3/3 [==============================] - ETA: 0s - loss: 0.0024 - accuracy: 1.00 - 0s 7ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 6/50
3/3 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 1.00 - 0s 7ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 7/50
3/3 [==============================] - ETA: 0s - loss: 0.0017 - accuracy: 1.00 - 0s 7ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 8/50
3/3 [=================

In [99]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [101]:
# cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
# for filename in os.listdir(f'b_cry/{g}'):
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=5)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
plt.savefig(f'live_data/output.png')
plt.clf()

<Figure size 576x576 with 0 Axes>

In [102]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [124]:
file = open('predict.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
# genres = 'discomfort belly_pain burping hungry'.split()
# for g in genres:
#     for filename in os.listdir(f'b_cry/{g}'):
# songname = f'b_cry/{g}/{filename}'
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=5)
rmse = librosa.feature.rms(y=y)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
to_append = f'output.wav {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
for e in mfcc:
    to_append += f' {np.mean(e)}'
# to_append += f' {model.predict(live_data.iloc[0])}'
#to_append += f' live_data_genre'
file = open('predict.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())
live_data = pd.read_csv('predict.csv')
live_data = live_data.drop(['filename'],axis=1)


In [139]:
#print(live_data.iloc[0][0:26])
Xnew = np.array([live_data.iloc[0][0:26]])
ynew = model.predict_classes(Xnew)
print("Predicted=%s" % (ynew[0]))
#live_data = data.drop(['filename'],axis=1)
#print(model.predict(live_data.iloc[:, -1]))
# print(type(live_data.iloc[0]))
#print(model.predict_classes(live_data.iloc[0]))
# print(live_data.iloc[0])

Predicted=3
